<a href="https://colab.research.google.com/github/devloper13/SiameseNetworkProject/blob/master/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from builtins import input

#import system things
from tensorflow.examples.tutorials.mnist import input_data # for data
import tensorflow as tf
import numpy as np
import os
import keras


# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import tensorflow as tf
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
class Siamese:

    # Create model
    def __init__(self):
        self.x1 = tf.placeholder(tf.float32, [None, 64881])
        self.x2 = tf.placeholder(tf.float32, [None, 64881])

        with tf.variable_scope("Siamese") as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)

        # Create loss
        self.y_ = tf.placeholder(tf.float32, [None])
        #self.y_ = tf.placeholder(tf.int32, [None])
        self.loss = self.cosineLoss()

    def network(self, x):
        
        x = tf.reshape(x,shape=[-1,1,64881,1])
        activated_conv1 = self.conv_layer('conv_1',x,3)
        maxpool1 = self.maxpool_layer('maxp_1',activated_conv1)
        
        flattened_conv = tf.layers.flatten(maxpool1)   #To be removed
        activated_fc1 = self.fc_layer( "fc1",flattened_conv, 128)
        #activated_fc2 = self.fc_layer("fc2",activated_fc1, 1024)
        #activated_fc3 = self.fc_layer("fc3",activated_fc2, 2)
        
        return activated_fc1
        
    def conv_layer(self,name,inputs,cur_channel):
        #print(inputs.get_shape())
        prev_channel = inputs.get_shape()[-1]
        #print(prev_channel)
        init = tf.variance_scaling_initializer(scale=2.0)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[1,10,prev_channel,cur_channel],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_channel],initializer = init)
        conv = tf.nn.conv2d(inputs,w,strides=[1,1,1,1],padding = "SAME")
        activation = tf.nn.relu(conv+b)
        return activation  
      
    def maxpool_layer(self,name,inputs):
        return tf.nn.max_pool(inputs,ksize=[1,1,100,1],strides=[1,1,100,1],padding="SAME")  
    
    def fc_layer(self,name,inputs,cur_layer):
        print(inputs.get_shape())
        prev_layer = inputs.get_shape()[-1]
        init = tf.truncated_normal_initializer(stddev=0.01)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[prev_layer,cur_layer],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_layer],initializer=init)
        activation = tf.matmul(inputs,w)+b
        return activation



        
    def cosineLoss(self):
        
        norms1 = tf.norm(self.o1,axis=1)
        norms2 = tf.norm(self.o2,axis=1)
        norm = tf.multiply(norms1,norms2)
        cosines = tf.reduce_sum(tf.multiply(self.o1,self.o2),axis=1)
        
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        
        
        C = tf.constant(0.4, name="C")
        
        pos = tf.multiply(labels_t, cosines, name="yi_x_cosine")
        
        neg = tf.multiply(labels_f, tf.maximum(tf.subtract(cosines,C),0), name="Nyi_x_C-cosine")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss
        

          

In [5]:
link="https://drive.google.com/open?id=1AycFi7m8_NYsK0zTuaAFrci0cLQdE51i"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('three_hash')  
with open('three_hash','rb') as f:
   three_hash_dict = pickle.load(f)
print(len(three_hash_dict))

64881


In [0]:
stop_words = set(stopwords.words('english')) 
def getThreeHash(text):
  vectorizer = CountVectorizer()
  tokenizer=vectorizer.build_tokenizer()
  
  hashes=""
  tokens=tokenizer(text)
  for token in tokens:
    if token not in stop_words:
      tokenModi="#"+token+"#"
      output = list(ngrams(tokenModi, 3))
      for a in output:
        hashes+=(''.join(a))+" "
  
  return(hashes)
# quest = "where do i find hot girls"
# hash_var = getThreeHash(quest)
#print(hash_var)

In [7]:
link="https://drive.google.com/open?id=1tpMcShoAyQ-uxJFz4SMDPzrg8glfbh1Z" #create shareable link of google drive file
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('QA.csv')  
ndata = pd.read_csv('QA.csv',error_bad_lines=False).values


1tpMcShoAyQ-uxJFz4SMDPzrg8glfbh1Z


In [0]:
data = ndata[ndata[:,4] == 'en']
data = data[:1500000]

In [0]:
def trainSiamese(quesVec,ansVec,siamese,sess,iteration=100):
  # setup siamese network
  siamese = Siamese();
  train_step = tf.train.GradientDescentOptimizer(0.01).minimize(siamese.loss)

  tf.initialize_all_variables().run()


  for step in range(iteration):
      batch_x1, batch_y1 = quesVec,np.array([1]*10)
#       print(batch_x1)
#       print(batch_y1)
      batch_x2, batch_y2 = ansVec,np.array([1]*10)
      batch_y = (batch_y1 == batch_y2).astype('float')

      _, loss_v = sess.run([train_step,siamese.loss], feed_dict={
                          siamese.x1: batch_x1,
                          siamese.x2: batch_x2,
                          siamese.y_: batch_y})
      
      
#       if np.isnan(loss_v):
#           print('Model diverged with loss = NaN')
#           quit()

#       if step % 10 == 0:
#           print ('step %d: loss %.3f' % (step, loss_v))

#       if step % 1000 == 0 and step > 0:
#           #saver.save(sess, './model')
#           embed = siamese.o1.eval({siamese.x1: mnist.test.images})
#           embed.tofile('embed.txt')


In [0]:
def vectorize(hashString,dictionary):
  
  vectorizer = CountVectorizer()
  tokenizer = vectorizer.build_tokenizer()
  vec = [0]*64881
  
  for token in tokenizer(hashString):
    try:
      vec[dictionary[token]] += 1
    except:
      pass
    
  return vec

def getRandomBatch(data,batchsize,dictionary,good_bad=0.5):
  
  goodSet = np.random.permutation(data.shape[0])[:int(batchsize*good_bad)]
  badSetQues = np.random.permutation(data.shape[0])[:batchsize - int(batchsize*good_bad)]
  badSetAns = np.random.permutation(data.shape[0])[:batchsize - int(batchsize*good_bad)]
  
  good_data = data[goodSet]
  bad_quest = data[badSetQues,1]
  bad_anser = data[badSetAns,3]
  questions = np.concatenate((good_data[:,1],bad_quest))
  answers = np.concatenate((good_data[:,3],bad_anser))
  
  label = np.array([1]*int(batchsize*good_bad) + [0]*(batchsize - int(batchsize*good_bad)))
  
  
  ques,ans = [],[]
  for d in questions:
    ques += [vectorize(getThreeHash(d.lower()),dictionary)]
  for i,d in enumerate(answers):
    
#     try:
#       print(goodSet[i])
#     except:
#       print(badSetAns[i - goodSet.shape[0]])
      
#     print(d)

    ans += [vectorize(getThreeHash(str(d).lower()),dictionary)]
  return np.array(ques),np.array(ans)
    
  

In [0]:
ques,ans = getRandomBatch(data,50,three_hash_dict)

In [43]:
ques.shape

(50, 64881)

In [25]:
# prepare data and tf.session
# mnist = input_data.read_data_sets('MNIST_data', one_hot=False)
tf.reset_default_graph()
sess = tf.InteractiveSession() 
vectorizer = CountVectorizer()
tokenizer=vectorizer.build_tokenizer()
rowcount = 0
qList,aList = [],[]
for row in data[0:100]:
  rowcount += 1
  quesVec,ansVec = [0]*64881,[0]*64881
  quesHash = getThreeHash(row[1].lower())
  ansHash = getThreeHash(row[3].lower())
#   print(quesHash)
  for token in tokenizer(quesHash.lower()):
     quesVec[three_hash_dict[token]] += 1
  for token in tokenizer(ansHash.lower()):
     ansVec[three_hash_dict[token]] += 1
  qList.append(quesVec)
  aList.append(ansVec)
  if rowcount % 10 == 0:    
    trainSiamese(qList,aList)
    qList,aList = [],[]
sess.close()
    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(?, 1947)
(?, 1947)
step 0: loss 9.297
step 10: loss 0.676
step 20: loss 0.385
step 30: loss 0.253
step 40: loss 0.169
step 50: loss 0.113
step 60: loss 0.075
step 70: loss 0.050
step 80: loss 0.034
step 90: loss 0.022


ValueError: ignored